In [2]:
%pip install --upgrade langchain langchain-community langgraph ollama langchain-ollama

Note: you may need to restart the kernel to use updated packages.


In [4]:
from IPython.display import display, Image
from langgraph.graph import StateGraph, START
from langchain_ollama import ChatOllama
import requests
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import MessagesState

from langgraph.prebuilt import ToolNode, tools_condition

from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages



In [5]:
class State(TypedDict):
    messages: Annotated[list,add_messages]

In [ ]:
%pip install -U duckduckgo-search

In [8]:
from langchain_community.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()
search.invoke("Who was Abhram Lincoln?")


"Abraham Lincoln[b] (February 12, 1809 – April 15, 1865) was the 16th president of the United States , serving from 1861 until his assassination in 1865. He led the United States through the American Civil War, defeating the Confederate States and playing a major role in the abolition of slavery. Abraham Lincoln was a member of the Whig Party and later a Republican. He believed that the government’s job was to do what a community of people could not do for themselves. One of his greatest preoccupations as a political thinker was the issue of self-governance and the promise and problems that could arise from it. The choice by some to allow the expansion of slavery was one such problem and was central to the American Civil War. Although opposed to slavery from the outset of his political career, Lincoln would not make its abolition a mainstay of his policy until several years into the war. Republican Party Learn the history of the Republican Party. How did Abraham Lincoln get into politi

In [9]:
from langchain_community.tools import DuckDuckGoSearchRun
def seach_duckduckgo(query: str) -> str:
    """Searches DuckDuckGo using langchain_community's DuckDuckGoSearchRun tool."""
    search = DuckDuckGoSearchRun()
    return search.invoke(query)

result = seach_duckduckgo("Who was Abraham Lincoln?")
print(result)




1 day ago - Abraham Lincoln (February 12, 1809 – April 15, 1865) was the 16th president of the United States , serving from 1861 until his assassination in 1865. He led the United States through the American Civil War, defeating the Confederate States and playing a major role in the abolition of slavery. 4 days ago - On April 14, 1865, Abraham Lincoln, the 16th president of the United States , was shot by John Wilkes Booth while attending the play Our American Cousin at Ford's Theatre in Washington, D.C. Shot in the head as he watched the play, Lincoln died of his wounds the following day at 7:22 a.m. in ... August 4, 2025 - Abraham Lincoln (/ˈlɪŋkən/ LINK-ən; February 12, 1809 – April 15, 1865) was an American politician who served as the 16th president of the United States from 1861 until his assassination in 1865 . He led the United States through the American Civil War, defeating the Confederate States ... 2 weeks ago - Abraham Lincoln was born on February 12, 1809, in a one-room l

In [45]:
def multiply(a: int, b: int) -> int:
    """Multiply a and b"""
    return a * b


def add(a: int, b: int) -> int:
    """
    add  a and b
    """
    return a + b

In [12]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2", temperature=0)


In [13]:
llm.invoke('hello').content

'Hello! How can I assist you today?'

In [ ]:
tools =[seach_duckduckgo, add, multiply]
llm_with_tools = llm.bind_tools(tools)


In [46]:
# Corrected chatbot function
def chatbot(state: State):
    return {"message": [llm_with_tools.invoke(state["messages"])]}

In [42]:
from langgraph.prebuilt import ToolNode, tools_condition

graph_builder = StateGraph(State)

# Define Nodes
graph_builder.add_node("assistant", chatbot)
graph_builder.add_node("tools", ToolNode(tools))

# Define Edges
graph_builder.add_edge(START, "assistant")
graph_builder.add_conditional_edges("assistant", tools_condition)
graph_builder.add_edge("tools", "assistant")
react_graph = graph_builder.compile()

In [ ]:
display(Image(react_graph.get_graph().draw_mermaid_png())) 

In [48]:
response = react_graph.invoke({"messages": [HumanMessage(content="what was Abraham Lincoln' birth year? add 2 to it and multiply by 3")]})
print(response['messages'])
    

[HumanMessage(content="what was Abraham Lincoln' birth year? add 2 to it and multiply by 3", additional_kwargs={}, response_metadata={}, id='7aa852a6-f14f-4bac-8063-867f1e26412b'), AIMessage(content='', additional_kwargs={}, response_metadata={}, id='4eba3a02-8056-4852-ac05-3c1a93cfe6f0'), AIMessage(content='', additional_kwargs={}, response_metadata={}, id='bca6302c-cb8c-4847-a0b0-9ff7eddc1c6f')]


In [ ]:
from m in response['messages']:
m.preetty_print()